# AquaMind – Pré-processamento e Modelagem

Objetivos:
1. Tratar dados (valores faltantes, outliers, normalização).
2. Criar features temporais.
3. Treinar e comparar 3 modelos (Regressão Linear, Árvore de Decisão, MLP).
4. Justificar escolha do modelo final.
5. Salvar e exportar o modelo.


In [27]:
# 📦 1. Importar libs
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 📦 Imports para classificação
from sklearn.linear_model    import LogisticRegression
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier
from sklearn.metrics         import accuracy_score

In [28]:
# 📥 2. Carregar dados
df = pd.read_csv('../data/sensor_data.csv', parse_dates=['timestamp'])
# 2.1 Remover linhas sem umidade ou temperatura
df = df.dropna(subset=['soil_moisture','temperature'])

In [29]:
# 🔧 3. Tratamento de outliers (remover 3σ)
from scipy import stats
z = np.abs(stats.zscore(df[['soil_moisture','temperature']]))
df = df[(z < 3).all(axis=1)]

In [30]:
# 🔨 4. Engenharia de features
df['hour']       = df['timestamp'].dt.hour
df['dayofyear']  = df['timestamp'].dt.dayofyear
df['roll_moist'] = df['soil_moisture'].rolling(window=3, min_periods=1).mean()

In [31]:
# 🔢 5. Definir X e y (prevendo próxima umidade)
features = ['temperature','hour','dayofyear','roll_moist']
X = df[features]
y = df['soil_moisture'].shift(-1).fillna(method='ffill')

C:\Users\Pichau\AppData\Local\Temp\ipykernel_9660\1514059793.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = df['soil_moisture'].shift(-1).fillna(method='ffill')


In [32]:
# ⚖️ 6. Escalonamento
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, '../models/aquamind_scaler.joblib')

['../models/aquamind_scaler.joblib']

In [33]:
# 🔀 7. Dividir treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [34]:

# 🧪 8. Configurar validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [35]:
# 📊 9. Definir modelos
models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree':     DecisionTreeRegressor(max_depth=5, random_state=42),
    'MLPRegressor':     MLPRegressor(hidden_layer_sizes=(50,50), max_iter=500, random_state=42)
}

In [36]:
# 🔍 10. Avaliar modelos via cross-val
results = []
for name, mdl in models.items():
    scores = cross_val_score(
        mdl, X_train, y_train, cv=kf, scoring='neg_root_mean_squared_error'
    )
    results.append({
        'model': name,
        'CV_RMSE_mean': -scores.mean(),
        'CV_RMSE_std':  scores.std()
    })
res_df = pd.DataFrame(results).sort_values('CV_RMSE_mean')
print(res_df)


c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the 

              model  CV_RMSE_mean  CV_RMSE_std
0  LinearRegression      2.246268     0.131002
1      DecisionTree      2.450051     0.160048
2      MLPRegressor      2.532077     0.221420


c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [37]:
# 📈 11. Teste final no melhor modelo
best_name  = res_df.iloc[0]['model']
best_model = models[best_name]
best_model.fit(X_train, y_train)
y_pred     = best_model.predict(X_test)

# Calcular RMSE manualmente
mse_test   = mean_squared_error(y_test, y_pred)
rmse_test  = np.sqrt(mse_test)
r2_test    = r2_score(y_test, y_pred)

print(f"Melhor modelo: {best_name}")
print(f"Teste RMSE: {rmse_test:.3f}")
print(f"Teste R²:   {r2_test:.3f}")


Melhor modelo: LinearRegression
Teste RMSE: 2.456
Teste R²:   0.880


# 📝 12. Justificativa da escolha
# Optamos por **{best_name}** pois apresentou o menor RMSE médio em validação cruzada,
# boa estabilidade (baixo desvio padrão) e equilíbrio entre performance e interpretabilidade.

In [38]:

# 💾 13. Salvar modelo treinado
joblib.dump(best_model, '../models/aquamind_model.joblib')

['../models/aquamind_model.joblib']

# Modelo de Acuracia
## 14. Classificação binária: necessidade de irrigação

Aqui vamos rotular como **1 (irrigar)** todas as leituras com umidade abaixo de 30%  
e **0 (não irrigar)** caso contrário. Depois treinamos três classificadores  
e comparamos a acurácia média em validação cruzada.

In [39]:
# 14.1 Criar y_binário
# threshold = 30% de umidade => irrigar
y_bin = (df['soil_moisture'] < 30).astype(int)


In [40]:
# As mesmas features já escalonadas em X_scaled
X_cls = X_scaled.copy()


In [41]:
# 14.2 Dividir em treino/teste
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_bin, test_size=0.2, random_state=42, stratify=y_bin
)

In [42]:
# 14.3 Definir os classificadores
classifiers = {
    'LogisticRegression':    LogisticRegression(max_iter=200, random_state=42),
    'DecisionTree':          DecisionTreeClassifier(max_depth=5, random_state=42),
    'RandomForest':          RandomForestClassifier(n_estimators=100, random_state=42)
}


In [43]:
# 14.4 Avaliar com validação cruzada (acurácia)
kf_cls = KFold(n_splits=5, shuffle=True, random_state=42)
cls_results = []
for name, clf in classifiers.items():
    scores = cross_val_score(clf, Xc_train, yc_train,
                             cv=kf_cls, scoring='accuracy')
    cls_results.append({
        'model':        name,
        'CV_Accuracy':  scores.mean(),
        'Std':          scores.std()
    })
cls_df = pd.DataFrame(cls_results).sort_values('CV_Accuracy', ascending=False)
display(cls_df)

,model,CV_Accuracy,Std
0,LogisticRegression,0.957182,0.024655
2,RandomForest,0.951610,0.015984
1,DecisionTree,0.944185,0.016497


## 15. Teste final do melhor classificador

Treinar no conjunto completo de treino e avaliar a acurácia no `Xc_test`/`yc_test`.


In [44]:
# 15.1 Selecionar melhor pelo CV
best_cls_name = cls_df.iloc[0]['model']
best_clf     = classifiers[best_cls_name]

# 15.2 Treinar e prever
best_clf.fit(Xc_train, yc_train)
yc_pred = best_clf.predict(Xc_test)

# 15.3 Calcular acurácia
acc_test = accuracy_score(yc_test, yc_pred)
print(f"Melhor classificador: {best_cls_name}")
print(f"Acurácia de teste:    {acc_test:.3f}")


Melhor classificador: LogisticRegression
Acurácia de teste:    0.956
